In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS CALC CLASSIFIER USING MAMMOGRAM PATCHES IN BCDR
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
import time


vid='v003_1'

In [2]:
# Create CNN

IMG_SIZE=224

from tensorflow.keras.applications.xception import Xception
base_model=Xception(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(512)(x) 
x=ReLU()(x)
preds=Dense(1,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

 block4_sepconv1_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 28, 28, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 28, 28, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 14, 14, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 14, 14, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 14, 14, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 14, 14, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 14, 14, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 14, 14, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 14, 14, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 14, 14, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_10 (A

In [4]:
len(model.layers)

139

In [5]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy']) 

In [6]:
# Image preprocessing and data augmentation

batch_size=8

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode="wrap",
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 90,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('mass_ben_mal_class02/train0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class02/valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)


Found 532 images belonging to 2 classes.
Found 134 images belonging to 2 classes.


In [7]:
#Callbacks
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [8]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mass_ben_mal_class_xception_weights_"+vid+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=1,
                   callbacks = callbacks)  

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12392\1152529500.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/3
66/66 [==============================] - 56s 776ms/step - loss: 0.5597 - accuracy: 0.7424 - val_loss: 0.4366 - val_accuracy: 0.7734 - lr: 0.0010
Epoch 2/3
66/66 [==============================] - 50s 761ms/step - loss: 0.5644 - accuracy: 0.7385 - val_loss: 0.3948 - val_accuracy: 0.7734 - lr: 0.0010
Epoch 3/3
66/66 [==============================] - 51s 769ms/step - loss: 0.4512 - accuracy: 0.7786 - val_loss: 0.3678 - val_accuracy: 0.8203 - lr: 0.0010


In [9]:
#Training stage 2
for layer in model.layers[:90]:
    layer.trainable=False
for layer in model.layers[90:]:  #last ~33% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=1,
                   callbacks = callbacks)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12392\81111685.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/10
66/66 [==============================] - 104s 1s/step - loss: 0.4924 - accuracy: 0.7958 - val_loss: 0.3252 - val_accuracy: 0.8594 - lr: 1.0000e-04
Epoch 2/10
66/66 [==============================] - 92s 1s/step - loss: 0.4350 - accuracy: 0.8187 - val_loss: 0.3049 - val_accuracy: 0.8516 - lr: 1.0000e-04
Epoch 3/10
66/66 [==============================] - 91s 1s/step - loss: 0.3990 - accuracy: 0.8168 - val_loss: 0.2675 - val_accuracy: 0.8984 - lr: 1.0000e-04
Epoch 4/10
66/66 [==============================] - 89s 1s/step - loss: 0.3575 - accuracy: 0.8435 - val_loss: 0.2761 - val_accuracy: 0.8750 - lr: 1.0000e-04
Epoch 5/10
66/66 [==============================] - 89s 1s/step - loss: 0.3481 - accuracy: 0.8416 - val_loss: 0.2830 - val_accuracy: 0.8906 - lr: 1.0000e-04
Epoch 6/10
66/66 [==============================] - 91s 1s/step - loss: 0.3663 - accuracy: 0.8588 - val_loss: 0.2571 - val_accuracy: 0.8828 - lr: 1.0000e-04
Epoch 7/10
66/66 [==============================] - 88s 1

In [10]:
#Training stage 3
#time.sleep(360)
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
callbacks = get_callbacks(name_weights = name_weights, patience_lr=5)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/150


C:\Users\Administrator\AppData\Local\Temp\ipykernel_12392\2279829289.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


66/66 [==============================] - 230s 3s/step - loss: 0.3480 - accuracy: 0.8454 - val_loss: 0.2877 - val_accuracy: 0.8984 - lr: 1.0000e-05
Epoch 2/150
66/66 [==============================] - 203s 3s/step - loss: 0.3389 - accuracy: 0.8607 - val_loss: 0.3400 - val_accuracy: 0.8750 - lr: 1.0000e-05
Epoch 3/150
66/66 [==============================] - 204s 3s/step - loss: 0.3659 - accuracy: 0.8473 - val_loss: 0.3140 - val_accuracy: 0.8906 - lr: 1.0000e-05
Epoch 4/150
66/66 [==============================] - 204s 3s/step - loss: 0.3465 - accuracy: 0.8550 - val_loss: 0.3355 - val_accuracy: 0.8984 - lr: 1.0000e-05
Epoch 5/150
66/66 [==============================] - 210s 3s/step - loss: 0.3334 - accuracy: 0.8664 - val_loss: 0.3403 - val_accuracy: 0.8828 - lr: 1.0000e-05
Epoch 6/150
66/66 [==============================] - 205s 3s/step - loss: 0.3149 - accuracy: 0.8721 - val_loss: 0.3455 - val_accuracy: 0.8672 - lr: 1.0000e-05
Epoch 7/150
66/66 [==============================] - 204s 

In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class02/valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(valid_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(valid_generator.classes, Y_pred)
prec=precision_score(valid_generator.classes, y_pred,pos_label=1)
rec=recall_score(valid_generator.classes, y_pred,pos_label=1)
spe=recall_score(valid_generator.classes, y_pred,pos_label=0)
f1s=f1_score(valid_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(valid_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))

print('Sensitivity: ', rec)
print('Specificity: ', spe)
print('Precision score:', prec)
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('F1 score:', f1s)
print('MCC score:', mcc)
print('ROC AUC score:', ras)

Found 134 images belonging to 2 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_12392\3759042783.py:13: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(valid_generator)


Confusion Matrix:
[[84  7]
 [ 8 35]]
Sensitivity:  0.813953488372093
Specificity:  0.9230769230769231
Precision score: 0.8333333333333334
Accuracy: 0.8880597014925373
F1 score: 0.8235294117647058
MCC score: 0.7416888893230639
ROC AUC score: 0.9511883465371838


In [13]:
#Print estimation result for ensemble model
print(Y_pred)
print(valid_generator.classes)

[[5.06794429e-04]
 [2.22854674e-01]
 [1.05731569e-01]
 [1.29672559e-02]
 [1.54568609e-02]
 [9.79682207e-01]
 [2.70478372e-02]
 [9.81683254e-01]
 [3.70434046e-01]
 [5.47832064e-03]
 [6.40285667e-03]
 [7.68535808e-02]
 [9.48641121e-01]
 [4.55708895e-03]
 [5.90762973e-01]
 [3.34282637e-01]
 [1.68715231e-02]
 [2.86051691e-01]
 [3.37311745e-01]
 [1.58321902e-01]
 [5.93912415e-02]
 [3.05092335e-01]
 [1.21427707e-01]
 [1.14421397e-01]
 [8.26330334e-02]
 [8.05875123e-01]
 [3.05947699e-02]
 [6.09979639e-03]
 [1.29098207e-01]
 [7.62363598e-02]
 [1.71448261e-01]
 [6.35413945e-01]
 [3.30768436e-01]
 [2.95439780e-01]
 [8.20882618e-02]
 [1.56991975e-03]
 [2.49639004e-01]
 [9.49507728e-02]
 [5.32723079e-03]
 [3.31891328e-03]
 [1.42037589e-02]
 [2.69883811e-01]
 [3.55234891e-02]
 [5.13579696e-02]
 [3.52207595e-03]
 [4.61846709e-01]
 [3.56817901e-01]
 [2.54419833e-01]
 [5.87250479e-02]
 [1.20737635e-01]
 [2.84063420e-03]
 [6.70847073e-02]
 [8.67329091e-02]
 [7.85845816e-02]
 [2.66198870e-02]
 [3.210189

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('mass_ben_mal_class02/test', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)


# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(test_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(test_generator.classes, Y_pred)
prec=precision_score(test_generator.classes, y_pred,pos_label=1)
rec=recall_score(test_generator.classes, y_pred,pos_label=1)
spe=recall_score(test_generator.classes, y_pred,pos_label=0)
f1s=f1_score(test_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(test_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))

print('Sensitivity: ', rec)
print('Specificity: ', spe)
print('Precision score:', prec)
print('Accuracy:', accuracy_score(test_generator.classes, y_pred))
print('F1 score:', f1s)
print('MCC score:', mcc)
print('ROC AUC score:', ras)

Found 169 images belonging to 2 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_12392\3439002409.py:15: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator)


Confusion Matrix:
[[98 15]
 [19 37]]
Sensitivity:  0.6607142857142857
Specificity:  0.8672566371681416
Precision score: 0.7115384615384616
Accuracy: 0.7988165680473372
F1 score: 0.6851851851851851
MCC score: 0.5384540373943631
ROC AUC score: 0.8511378002528445


In [15]:
#Print estimation result for ensemble model
print(Y_pred)
print(test_generator.classes)

[[4.59553361e-01]
 [2.81136841e-01]
 [2.09634498e-01]
 [1.02638535e-01]
 [1.04024960e-02]
 [9.80843231e-03]
 [1.15241922e-01]
 [1.19133499e-02]
 [1.32228034e-02]
 [5.27925324e-04]
 [1.01180878e-02]
 [2.52058394e-02]
 [8.03932697e-02]
 [1.05070740e-01]
 [1.07177105e-02]
 [4.28901723e-04]
 [7.57188320e-01]
 [5.92114806e-01]
 [1.47532416e-03]
 [3.26317817e-01]
 [8.36129844e-01]
 [2.26318594e-02]
 [7.69989043e-02]
 [6.21401906e-01]
 [3.22931982e-03]
 [8.69084056e-03]
 [1.35229668e-02]
 [4.20773596e-01]
 [8.35779309e-01]
 [2.07398593e-01]
 [2.75311079e-02]
 [7.40632355e-01]
 [1.29850404e-02]
 [3.95902544e-01]
 [3.05045456e-01]
 [9.55586508e-03]
 [7.05795661e-02]
 [4.88436371e-01]
 [1.20722316e-02]
 [7.05767334e-01]
 [7.54784569e-02]
 [7.69952638e-03]
 [3.72382887e-02]
 [5.36025800e-02]
 [2.84909278e-01]
 [1.79614890e-02]
 [1.65748484e-02]
 [7.93744251e-03]
 [5.31244278e-01]
 [6.45436486e-03]
 [1.98766403e-02]
 [2.17855975e-01]
 [3.12016346e-02]
 [5.16142230e-03]
 [1.74203495e-04]
 [4.692837